In [14]:
import pandas as pd
import numpy as np

### Merge dataset

In [24]:
# df1 = pd.read_csv("dataset/analisis_essay.csv")
# df1['dataset'] = 'analisis_essay'
# df2 = pd.read_csv("dataset/asap.csv")
# df2['dataset'] = 'asap'
# df3 = pd.read_csv("dataset/cunlp.csv")
# df3['dataset'] = 'cunlp'
# df4 = pd.read_csv("dataset/sag.csv")
# df4['dataset'] = 'sag'
# df5 = pd.read_csv("dataset/sci.csv")
# df5['dataset'] = 'sci'
# df6 = pd.read_csv("dataset/stita.csv")
# df6['dataset'] = 'stita'

# # add null value to all dataframe that doesn't have question column
# for df in [df2, df3, df6]:
#     df['question'] = pd.NA

# df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
# df.shape

# df.to_csv("dataset/aes_dataset.csv", index=False)

### Preprocessing

In [4]:
df = pd.read_csv("dataset/aes_dataset.csv")
df.head()

,question,reference_answer,answer,score,dataset
0,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sumber tenaga, pemanis alami, menjaga sistem i...",27.0,analisis_essay
1,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sebagai sumber energi, pemanis alami, menjaga ...",21.0,analisis_essay
2,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,1. Sebagai energi. 2. Sebagai memperlancaar pe...,42.0,analisis_essay
3,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"untuk membuat kenyang, agar tidak lapar, agar ...",18.0,analisis_essay
4,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,Karbohidrat mempunyai peran penting untuk pros...,82.0,analisis_essay


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22406 entries, 0 to 22405
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  22406 non-null  object 
 2   answer            22406 non-null  object 
 3   score             22406 non-null  float64
 4   dataset           22406 non-null  object 
dtypes: float64(1), object(4)
memory usage: 875.4+ KB


In [5]:
df['dataset'].value_counts()

dataset
asap              17043
sag                2558
analisis_essay     2162
stita               333
cunlp               171
sci                 139
Name: count, dtype: int64

In [ ]:
# convert NaN to None
df['question'] = df['question'].replace({np.nan: None})